In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Teorema da máxima transferência de potência
---

Para estudarmos o teorema da máxima transferência de potência, supomos a posibilidade de alteração da impedância de carga, aqui denominada *resistência de carga* ($RL$).

Queremos estudar a potência entregue à carga à partir de um *modelo equivalente de Thevenin* de fonte. Para isso utilizamos a equação
\begin{align}
P_L = \dfrac{1}{2} \cdot \dfrac{V_{pk}^2 \cdot R_L}{(R_s + R_L)^2 + (X_s + X_L)^2}
\end{align}

Na equação $P_L$ é a potência de carga. Em nosso caso, assumimos que ambas impedâncias são estritamente reais. Ou seja, os termos $X_s$ e $X_L$ são iguais a zero. O que facilita nosso estudo.

In [ ]:
RL_MAX = 300 # resitência de carga máxima calculada.

Vpk = 1 # tensão de pico da fonte de sinal
RL = np.linspace(0, RL_MAX, 100) # variação da resitência de carga
Rs = 50 # resistência de fonte

Vo = Vpk * RL / (Rs + RL) # cálculo da tensão entregue à carga carga
PL = 0.5 * ((Vpk**2) * RL) / ((Rs + RL)**2) # calculo da potência entregue à carga

In [ ]:
fig = plt.figure(figsize=(12,4))

# plot da variação da tensão entregue à carga
# em relação à resitência de carga
plt.subplot(121)
plt.plot(RL,Vo)
plt.ylim(0,Vpk)
plt.xlim(0,RL_MAX)
plt.grid(True)

# plot da variação da potência entregue à carga
# em relação à resitência de carga
plt.subplot(122)
plt.plot(RL,PL)
plt.vlines(Rs, 0, np.max(PL)*(1.05), "r")
plt.ylim(0,np.max(PL)*(1.05))
plt.xlim(0,RL_MAX)
plt.grid(True)

## Exemplo adaptação de impedância
---

Abaixo seguimos com o exemplo numérico do exemplo mostrado na apostila. Aqui utilizamos o notebook python para proceder com o cálculo numérico dos componentes, e posteriormente fazemos a simulação (no simulador ngspice) do circuito com os valores calculados.

**1.** Desejamos calcular os componentes $C$ e $L_P$ da figura 1.7a. Em nosso exemplo temos que $Z_s=50\Omega$ e $Z_{L,P}=100\Omega$. Suponha que a rede deva adaptar a fonte à carga em uma frequência $f=f_0=1KHz$.

In [ ]:
# Iniciação das variáveis (componentes) coniecidos do circuito
Rs = 50
RLP = 100
RLS = 50

f0 = 1000
W0 = 2*np.pi*f0

Queremos encontrar os valores de $C$ e $L_P$, para isso, transformamos o circuito 1.7a em um circuito que conhecemos e sabemos calcular (1.7b). Dessa maneira, podemos igualar o valor da resistência de carga $R_{L,S}$ à $R_s$. Assim, é possível calcular o fator de qualidade necessário para o circuito de adaptação, com o uso da equação
$$
R_{L,S} = R_{L,P} = \dfrac{1}{1+Q^2} = 50\Omega
$$
Resolvendo a equação para $Q$, temos que
$$
Q = \sqrt{\dfrac{R_{L,P}}{R_S}-1}
$$

In [ ]:
Q = np.sqrt((RLP/Rs)-1)
print(Q)

E calculamos $L_S$ com a equação do fator de qualidade para o circuito ressonador em série...
$$
Q = \dfrac{\omega_0L_S}{R_{L,S}}
$$

In [ ]:
LS = Q*RLS/W0
print(LS)

Finalmente, calculamos o valor da capacitância utilizando a fórmula da frequência ressonante
$$
\omega_0 = \dfrac{1}{\sqrt{LC}}
$$

In [ ]:
C = 1/(LS*W0**2)
print(C)

Note que ao calcularmos a frequência ressonante com a utilização dos valores de $L_S$ e $C$, vemos que $f_0=999.999Hz$. O que não é exatamente $1KHz$. Esse resultado é devido ao erro numérico referente às operações. É Normal que haja esse tipo de erro, o importante é manter esse erro "_controlado_".

In [ ]:
W = 1/(np.sqrt(LS*C))
print(W/(2*np.pi))

Procedemos agora com o cálculo dos componentes do circuito "original" (figura 1.7a). Munido dos valores do circuito da figura 1.7b, procedemos com a transformação série-paralelo, de maneira a obter $L_S$ e $C$. Em todo caso, $C$ é o mesmo para ambos os circuitos, visto que apenas transformamos o circuito $LR$ paralelo em série.

Ou seja, utilizando
$$
L_S = L_P\cdot\dfrac{Q^2}{1+Q^2}
$$
para $Q=1$, que é o valor calculado anteriormente no circuito mostrado. Dessa forma temos que...

In [ ]:
LP = LS*2
print(LP)

Ao substituir os valores de $C$ e $L_P$ no circuito original (1.7b), e procedendo com o cálculo da impedância de entrada do circuito
$$
Z_A = \dfrac{1}{j\omega_0 C}+\dfrac{R_{L,P}\cdot j\omega_0 L_P}{R_{L,P} + j\omega_0 L_P}
$$

In [ ]:
ZA= ( 1/(1j*W0*C) ) + ( RLP*1j*W0*LP/(RLP + 1j*W0*LP) )
print(ZA.real)
print(ZA.imag)

Observe que a parte real do circuito é exatamente $Re\{Z_A\}=50\Omega$, e o componente imaginário da impedância é irrisório (na ordem de $10^{-17}$), mais uma vez devemos chamar a atenção para o *erro numérico* nesse ponto.

Ademais, vejamos como fica a impedância de entrada quando variamos a frequência. O gráfico é mostrado com o eixo de frequência em escala logarítmica.

In [ ]:
fig = plt.figure(figsize=(12,4))
f = np.logspace(2, 4, 1001)
W = 2*np.pi*f

Za= ( 1/(1j*W*C) ) + ( RLP*1j*W*LP/(1j*W*LP + RLP) )

# visualização da parte real da impedância
plt.subplot(121)
plt.semilogx(f, Za.real)
plt.xlim(1e2,1e4)
plt.ylim(0,1e2)
plt.grid(True)

# visualização da parte imaginária da impedância
plt.subplot(122)
plt.semilogx(f, Za.imag)
plt.xlim(1e2,1e4)
plt.ylim(-500,5e1)
plt.grid(True)

<font color='red'>**Agora que você tem todas as informações sobre o circuito e o exemplo 01 da apostila, vá até o simulador e execute o arquivo "exemplo04.sch", em posse desse arquivo você poderá estudar outras topologias e visualizar características importantes do circuito de adaptação de impedâncias.**</font>

---